In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This code is largely based off and inspired by the following notebooks:
1. https://www.kaggle.com/code/mathieuduverne/nfl-2026-simple-ensemble
2. https://www.kaggle.com/code/jakupymeraj/0-64-score-nfl-2026

I followed some of the suggested steps for tuning and incorporated and slightly different ensemble methodology, which was guided by visuals and was able to achieve slight improvements.

In [ ]:
import pandas as pd
import numpy as np
import torch
import pickle
import joblib
from pathlib import Path
import warnings
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# ============================================================================
# CONFIGURATION
# ============================================================================

class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    
    CATBOOST_MODEL_PATH = "/kaggle/input/nfl-2026-big-data-bowl/catboost_5fold_models.pkl"
    LSTM_MODEL_DIR = "/kaggle/input/nfl-big-data-bowl-2026-lstm"
    
    # Updated weights based on your suggestion
    ENSEMBLE_WEIGHTS = {
        'catboost': 0.6,
        'lstm': 0.4
    }
    
    # Position-specific weights (can be tuned based on validation)
    POSITION_WEIGHTS = {
        'Targeted Receiver': {'catboost': 0.5, 'lstm': 0.5},
        'Defensive Coverage': {'catboost': 0.65, 'lstm': 0.35},
        'Passer': {'catboost': 0.7, 'lstm': 0.3},
        'Other Route Runner': {'catboost': 0.6, 'lstm': 0.4},
        'default': {'catboost': 0.6, 'lstm': 0.4}
    }
    
    LSTM_N_FOLDS = 5
    LSTM_INPUT_DIM = 32
    LSTM_HIDDEN_DIM = 128
    LSTM_NUM_LAYERS = 2
    LSTM_DROPOUT = 0.3
    LSTM_MAX_FRAMES = 94
    LSTM_WINDOW_SIZE = 8
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3

# ============================================================================
# CATBOOST PIPELINE
# ============================================================================

def load_catboost_models(model_path):
    """Load CatBoost models from pickle file"""
    print(f"Loading CatBoost models from {model_path}...")
    with open(model_path, 'rb') as f:
        saved = pickle.load(f)
    return saved['models_x'], saved['models_y'], saved['features']

def engineer_catboost_features(df):
    """Reproduce feature engineering from CatBoost notebook"""
    df = df.copy()
    
    df['velocity_x'] = df['s'] * np.cos(np.radians(df['dir']))
    df['velocity_y'] = df['s'] * np.sin(np.radians(df['dir']))
    
    df['dist_to_ball'] = np.sqrt(
        (df['x'] - df['ball_land_x'])**2 + 
        (df['y'] - df['ball_land_y'])**2
    )
    
    df['angle_to_ball'] = np.arctan2(
        df['ball_land_y'] - df['y'],
        df['ball_land_x'] - df['x']
    )
    
    df['velocity_toward_ball'] = (
        df['velocity_x'] * np.cos(df['angle_to_ball']) + 
        df['velocity_y'] * np.sin(df['angle_to_ball'])
    )
    
    df['time_to_ball'] = df['num_frames_output'] / 10.0
    df['orientation_diff'] = np.abs(df['o'] - df['dir'])
    df['orientation_diff'] = np.minimum(df['orientation_diff'], 360 - df['orientation_diff'])
    
    df['role_targeted_receiver'] = (df['player_role'] == 'Targeted Receiver').astype(int)
    df['role_defensive_coverage'] = (df['player_role'] == 'Defensive Coverage').astype(int)
    df['role_passer'] = (df['player_role'] == 'Passer').astype(int)
    df['side_offense'] = (df['player_side'] == 'Offense').astype(int)
    
    height_parts = df['player_height'].str.split('-', expand=True)
    df['height_inches'] = height_parts[0].astype(float) * 12 + height_parts[1].astype(float)
    df['bmi'] = (df['player_weight'] / (df['height_inches']**2)) * 703
    
    df['acceleration_x'] = df['a'] * np.cos(np.radians(df['dir']))
    df['acceleration_y'] = df['a'] * np.sin(np.radians(df['dir']))
    df['distance_to_target_x'] = df['ball_land_x'] - df['x']
    df['distance_to_target_y'] = df['ball_land_y'] - df['y']
    df['speed_squared'] = df['s'] ** 2
    df['accel_magnitude'] = np.sqrt(df['acceleration_x']**2 + df['acceleration_y']**2)
    df['velocity_alignment'] = np.cos(df['angle_to_ball'] - np.radians(df['dir']))
    
    df['expected_x_at_ball'] = df['x'] + df['velocity_x'] * df['time_to_ball']
    df['expected_y_at_ball'] = df['y'] + df['velocity_y'] * df['time_to_ball']
    df['error_from_ball_x'] = df['expected_x_at_ball'] - df['ball_land_x']
    df['error_from_ball_y'] = df['expected_y_at_ball'] - df['ball_land_y']
    df['error_from_ball'] = np.sqrt(df['error_from_ball_x']**2 + df['error_from_ball_y']**2)
    
    df['momentum_x'] = df['player_weight'] * df['velocity_x']
    df['momentum_y'] = df['player_weight'] * df['velocity_y']
    df['kinetic_energy'] = 0.5 * df['player_weight'] * df['speed_squared']
    
    df['angle_diff'] = np.abs(df['o'] - np.degrees(df['angle_to_ball']))
    df['angle_diff'] = np.minimum(df['angle_diff'], 360 - df['angle_diff'])
    
    df['time_squared'] = df['time_to_ball'] ** 2
    df['dist_squared'] = df['dist_to_ball'] ** 2
    df['weighted_dist_by_time'] = df['dist_to_ball'] / (df['time_to_ball'] + 0.1)
    
    return df

def add_sequence_features_catboost(df):
    """Add temporal features for CatBoost"""
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    group_cols = ['game_id', 'play_id', 'nfl_id']
    
    for lag in [1, 2, 3, 4, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y', 's', 'a']:
            if col in df.columns:
                df[f'{col}_lag{lag}'] = df.groupby(group_cols)[col].shift(lag)
    
    for window in [3, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y', 's']:
            if col in df.columns:
                df[f'{col}_rolling_mean_{window}'] = df.groupby(group_cols)[col].rolling(window, min_periods=1).mean().reset_index(level=[0,1,2], drop=True)
                df[f'{col}_rolling_std_{window}'] = df.groupby(group_cols)[col].rolling(window, min_periods=1).std().reset_index(level=[0,1,2], drop=True)
    
    for col in ['velocity_x', 'velocity_y']:
        if col in df.columns:
            df[f'{col}_delta'] = df.groupby(group_cols)[col].diff()
    
    return df

def predict_catboost(models_x, models_y, features, test_input, test_template):
    """Generate CatBoost predictions"""
    print("Generating CatBoost predictions...")
    
    test_features = engineer_catboost_features(test_input)
    test_features = add_sequence_features_catboost(test_features)
    
    test_agg = test_features.groupby(['game_id', 'play_id', 'nfl_id']).last().reset_index()
    if 'frame_id' in test_agg.columns:
        test_agg = test_agg.drop('frame_id', axis=1)
    
    test_merged = test_template.merge(
        test_agg,
        on=['game_id', 'play_id', 'nfl_id'],
        how='left'
    )
    
    for col in features:
        if col not in test_merged.columns:
            test_merged[col] = 0
    
    X_test = test_merged[features].fillna(0).values
    
    pred_x = np.mean([model.predict(X_test) for model in models_x], axis=0)
    pred_y = np.mean([model.predict(X_test) for model in models_y], axis=0)
    
    predictions = pd.DataFrame({
        'id': (test_merged['game_id'].astype(str) + '_' + 
              test_merged['play_id'].astype(str) + '_' + 
              test_merged['nfl_id'].astype(str) + '_' + 
              test_merged['frame_id'].astype(str)),
        'x': pred_x,
        'y': pred_y
    })
    
    return predictions

# ============================================================================
# LSTM PIPELINE
# ============================================================================

class ImprovedLSTMRegressor(torch.nn.Module):
    """LSTM architecture identical to training notebook"""
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.3, max_frames_output=94):
        super().__init__()
        self.max_frames_output = max_frames_output
        
        self.lstm = torch.nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(64, 2 * max_frames_output)
        )
    
    def forward(self, x, output_lengths=None):
        lstm_out, _ = self.lstm(x)
        last_out = lstm_out[:, -1, :]
        all_outputs = self.fc(last_out)
        batch_size = all_outputs.shape[0]
        outputs = all_outputs.view(batch_size, self.max_frames_output, 2)
        return outputs

def load_lstm_models(models_dir, n_folds):
    """Load LSTM models and scalers from fold_X/ directories"""
    print(f"Loading LSTM models from {models_dir}...")
    
    models = []
    scalers = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for fold in range(n_folds):
        model_path = Path(models_dir) / f'fold_{fold+1}/lstm_model_fold.pt'
        scaler_path = Path(models_dir) / f'fold_{fold+1}/lstm_feature_scaler_fold.joblib'
        
        ckpt = torch.load(model_path, map_location=device)
        
        if isinstance(ckpt, dict) and 'state_dict' in ckpt:
            state_dict = ckpt['state_dict']
            cfg = ckpt.get('config', {})
            input_dim = cfg.get('input_dim', Config.LSTM_INPUT_DIM)
            hidden_dim = cfg.get('hidden_dim', Config.LSTM_HIDDEN_DIM)
            num_layers = cfg.get('num_layers', Config.LSTM_NUM_LAYERS)
            dropout = cfg.get('dropout', Config.LSTM_DROPOUT)
            max_frames = cfg.get('max_frames_output', Config.LSTM_MAX_FRAMES)
        else:
            state_dict = ckpt
            input_dim = Config.LSTM_INPUT_DIM
            hidden_dim = Config.LSTM_HIDDEN_DIM
            num_layers = Config.LSTM_NUM_LAYERS
            dropout = Config.LSTM_DROPOUT
            max_frames = Config.LSTM_MAX_FRAMES
        
        model = ImprovedLSTMRegressor(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            max_frames_output=max_frames
        )
        model.load_state_dict(state_dict)
        model.to(device)
        model.eval()
        models.append(model)
        
        scaler = joblib.load(scaler_path)
        scalers.append(scaler)
        
        print(f"Loaded fold {fold+1}")
    
    return models, scalers

def height_to_feet(height_str):
    try:
        ft, inches = map(int, height_str.split('-'))
        return ft + inches/12
    except:
        return None

def prepare_lstm_sequences(input_df, test_template, window_size):
    """Prepare LSTM sequences for inference"""
    print("Preparing LSTM sequences...")
    
    input_df = input_df.copy()
    input_df['player_height_feet'] = input_df['player_height'].map(height_to_feet)
    
    dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    delta_t = 0.1
    input_df['velocity_x'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.sin(dir_rad)
    input_df['velocity_y'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.cos(dir_rad)
    
    input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    input_df['is_receiver'] = (input_df['player_role'] == 'Receiver').astype(int)
    input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    
    mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
    input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
    input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
    
    current_date = datetime.now()
    input_df['age'] = input_df['player_birth_date'].apply(
        lambda x: (current_date - datetime.strptime(x, '%Y-%m-%d')).days // 365 if pd.notnull(x) else None
    )
    
    input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)
    input_df['force'] = mass_kg * input_df['a']
    
    input_df['rolling_mean_velocity_x'] = input_df.groupby(['game_id', 'play_id', 'nfl_id'])['velocity_x'].transform(
        lambda x: x.rolling(window=window_size, min_periods=1).mean()
    )
    input_df['rolling_std_acceleration'] = input_df.groupby(['game_id', 'play_id', 'nfl_id'])['a'].transform(
        lambda x: x.rolling(window=window_size, min_periods=1).std()
    )
    
    if all(col in input_df.columns for col in ['ball_land_x', 'ball_land_y']):
        ball_dx = input_df['ball_land_x'] - input_df['x']
        ball_dy = input_df['ball_land_y'] - input_df['y']
        input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
        input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
        input_df['closing_speed'] = (input_df['velocity_x'] * input_df['ball_direction_x'] +
                                     input_df['velocity_y'] * input_df['ball_direction_y'])
        input_df['estimated_time_to_ball'] = input_df['distance_to_ball'] / 20.0
        input_df['projected_time_to_ball'] = input_df['distance_to_ball'] / (np.abs(input_df['closing_speed']) + 0.1)
    
    input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    input_df['is_right'] = (input_df['play_direction'] == 'right').astype(int)
    input_df['is_left'] = (input_df['play_direction'] == 'left').astype(int)
    
    feature_cols = [
        'x','y','s','a','o','dir',
        'absolute_yardline_number',
        'player_height_feet','player_weight',
        'is_right','is_left',
        'velocity_x','velocity_y',
        'momentum_x','momentum_y',
        'is_offense','is_defense','is_receiver','is_coverage','is_passer',
        'age',
        'kinetic_energy','force',
        'rolling_mean_velocity_x','rolling_std_acceleration'
    ]
    if 'distance_to_ball' in input_df.columns:
        feature_cols += [
            'distance_to_ball','angle_to_ball','ball_direction_x','ball_direction_y',
            'closing_speed','estimated_time_to_ball','projected_time_to_ball'
        ]
    
    grouped_input = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
    target_groups = test_template[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
    
    sequences, sequence_ids = [], []
    
    for _, row in target_groups.iterrows():
        key = (row['game_id'], row['play_id'], row['nfl_id'])
        try:
            group_df = grouped_input.get_group(key)
        except KeyError:
            continue
        
        input_window = group_df.tail(window_size)
        
        if len(input_window) < window_size:
            pad_length = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_length), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True).reset_index(drop=True)
        
        seq = input_window[feature_cols].values
        
        if np.isnan(seq.astype(np.float32)).any():
            seq = np.nan_to_num(seq, nan=0.0)
        
        sequences.append(seq)
        
        last_frame_id = input_window['frame_id'].iloc[-1]
        sequence_ids.append({
            'game_id': key[0],
            'play_id': key[1],
            'nfl_id': key[2],
            'frame_id': last_frame_id
        })
    
    return sequences, sequence_ids

def predict_lstm(models, scalers, test_input, test_template):
    """Generate LSTM predictions"""
    print("Generating LSTM predictions...")
    
    sequences, seq_ids = prepare_lstm_sequences(
        test_input, 
        test_template, 
        Config.LSTM_WINDOW_SIZE
    )
    
    X_test_unscaled = np.array(sequences, dtype=object)
    test_meta = pd.DataFrame(seq_ids)
    
    x_last = np.array([seq[-1, 0] for seq in X_test_unscaled], dtype=np.float32)
    y_last = np.array([seq[-1, 1] for seq in X_test_unscaled], dtype=np.float32)
    test_meta['x_last'] = x_last
    test_meta['y_last'] = y_last
    
    per_model_abs = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i, (model, scaler) in enumerate(zip(models, scalers)):
        scaled = np.array([scaler.transform(s) for s in X_test_unscaled], dtype=object)
        stacked = np.stack(scaled.astype(np.float32))
        
        test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(stacked))
        loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False)
        
        dx_list, dy_list = [], []
        with torch.no_grad():
            for (batch,) in loader:
                batch = batch.to(device)
                out = model(batch)
                dx_list.append(out[:, :, 0].cpu().numpy())
                dy_list.append(out[:, :, 1].cpu().numpy())
        
        dx_cum = np.vstack(dx_list)
        dy_cum = np.vstack(dy_list)
        
        abs_all_x = x_last[:, None] + dx_cum
        abs_all_y = y_last[:, None] + dy_cum
        
        per_model_abs.append((abs_all_x, abs_all_y))
    
    M = len(per_model_abs)
    N = len(test_meta)
    max_h = per_model_abs[0][0].shape[1]
    
    pad_x = np.full((M, N, max_h), np.nan, dtype=np.float32)
    pad_y = np.full((M, N, max_h), np.nan, dtype=np.float32)
    
    for m, (ax, ay) in enumerate(per_model_abs):
        h = ax.shape[1]
        pad_x[m, :, :h] = ax
        pad_y[m, :, :h] = ay
    
    ens_x = np.nanmean(pad_x, axis=0)
    ens_y = np.nanmean(pad_y, axis=0)
    
    out_rows = []
    for i, seq_info in test_meta.iterrows():
        game_id = int(seq_info['game_id'])
        play_id = int(seq_info['play_id'])
        nfl_id = int(seq_info['nfl_id'])
        
        frame_ids = test_template[
            (test_template['game_id'] == game_id) &
            (test_template['play_id'] == play_id) &
            (test_template['nfl_id'] == nfl_id)
        ]['frame_id'].sort_values().tolist()
        
        for t, frame_id in enumerate(frame_ids):
            if t < ens_x.shape[1]:
                px = ens_x[i, t]
                py = ens_y[i, t]
            else:
                px = ens_x[i, -1]
                py = ens_y[i, -1]
            
            out_rows.append({
                'id': f"{game_id}_{play_id}_{nfl_id}_{frame_id}",
                'x': px,
                'y': py
            })
    
    predictions = pd.DataFrame(out_rows)
    return predictions

# ============================================================================
# ADVANCED ENSEMBLE METHODS
# ============================================================================

def create_position_adaptive_ensemble(catboost_pred, lstm_pred, test_input, weights_config):
    """
    Create ensemble with position-specific weights
    Different player roles may benefit from different model weightings
    """
    print("Creating position-adaptive ensemble...")
    
    # Extract player roles from test_input
    player_info = test_input[['game_id', 'play_id', 'nfl_id', 'player_role']].drop_duplicates()
    
    # Merge predictions with player info
    catboost_with_role = catboost_pred.copy()
    lstm_with_role = lstm_pred.copy()
    
    # Extract game_id, play_id, nfl_id from id column
    catboost_with_role[['game_id', 'play_id', 'nfl_id', 'frame_id']] = catboost_with_role['id'].str.split('_', expand=True).astype(int)
    lstm_with_role[['game_id', 'play_id', 'nfl_id', 'frame_id']] = lstm_with_role['id'].str.split('_', expand=True).astype(int)
    
    # Merge with player info
    catboost_with_role = catboost_with_role.merge(player_info, on=['game_id', 'play_id', 'nfl_id'], how='left')
    lstm_with_role = lstm_with_role.merge(player_info, on=['game_id', 'play_id', 'nfl_id'], how='left')
    
    # Apply position-specific weights
    ensemble = []
    for idx, row in catboost_with_role.iterrows():
        role = row['player_role']
        weights = weights_config.get(role, weights_config['default'])
        
        w_cat = weights['catboost']
        w_lstm = weights['lstm']
        total_weight = w_cat + w_lstm
        
        lstm_row = lstm_with_role.iloc[idx]
        
        ens_x = (row['x'] * w_cat + lstm_row['x'] * w_lstm) / total_weight
        ens_y = (row['y'] * w_cat + lstm_row['y'] * w_lstm) / total_weight
        
        ensemble.append({
            'id': row['id'],
            'x': np.clip(ens_x, Config.FIELD_X_MIN, Config.FIELD_X_MAX),
            'y': np.clip(ens_y, Config.FIELD_Y_MIN, Config.FIELD_Y_MAX),
            'player_role': role
        })
    
    return pd.DataFrame(ensemble)

def create_confidence_weighted_ensemble(catboost_pred, lstm_pred, weights):
    """
    Weighted ensemble with uncertainty consideration
    Clips to field boundaries
    """
    catboost_pred = catboost_pred.sort_values('id').reset_index(drop=True)
    lstm_pred = lstm_pred.sort_values('id').reset_index(drop=True)
    
    w_cat = weights['catboost']
    w_lstm = weights['lstm']
    total_weight = w_cat + w_lstm
    
    # Calculate prediction variance as a proxy for uncertainty
    x_variance = np.abs(catboost_pred['x'] - lstm_pred['x'])
    y_variance = np.abs(catboost_pred['y'] - lstm_pred['y'])
    
    # Adaptive weighting: when models disagree, rely more on the stronger model
    adaptive_w_cat = w_cat + 0.1 * (x_variance + y_variance) / (x_variance + y_variance + 1)
    adaptive_w_lstm = 1 - adaptive_w_cat
    
    ensemble = pd.DataFrame({
        'id': catboost_pred['id'],
        'x': (catboost_pred['x'] * adaptive_w_cat + lstm_pred['x'] * adaptive_w_lstm),
        'y': (catboost_pred['y'] * adaptive_w_cat + lstm_pred['y'] * adaptive_w_lstm),
        'uncertainty': x_variance + y_variance
    })
    
    ensemble['x'] = np.clip(ensemble['x'], Config.FIELD_X_MIN, Config.FIELD_X_MAX)
    ensemble['y'] = np.clip(ensemble['y'], Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
    
    return ensemble

# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def plot_prediction_distributions(catboost_pred, lstm_pred, ensemble_pred):
    """Visualize prediction distributions across models"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle('Model Prediction Distributions', fontsize=16, fontweight='bold')
    
    # X coordinate distributions
    axes[0, 0].hist(catboost_pred['x'], bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[0, 0].set_title('CatBoost X Predictions')
    axes[0, 0].set_xlabel('X Coordinate')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].axvline(catboost_pred['x'].mean(), color='red', linestyle='--', label=f'Mean: {catboost_pred["x"].mean():.2f}')
    axes[0, 0].legend()
    
    axes[0, 1].hist(lstm_pred['x'], bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[0, 1].set_title('LSTM X Predictions')
    axes[0, 1].set_xlabel('X Coordinate')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].axvline(lstm_pred['x'].mean(), color='red', linestyle='--', label=f'Mean: {lstm_pred["x"].mean():.2f}')
    axes[0, 1].legend()
    
    axes[0, 2].hist(ensemble_pred['x'], bins=50, alpha=0.7, color='purple', edgecolor='black')
    axes[0, 2].set_title('Ensemble X Predictions')
    axes[0, 2].set_xlabel('X Coordinate')
    axes[0, 2].set_ylabel('Frequency')
    axes[0, 2].axvline(ensemble_pred['x'].mean(), color='red', linestyle='--', label=f'Mean: {ensemble_pred["x"].mean():.2f}')
    axes[0, 2].legend()
    
    # Y coordinate distributions
    axes[1, 0].hist(catboost_pred['y'], bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[1, 0].set_title('CatBoost Y Predictions')
    axes[1, 0].set_xlabel('Y Coordinate')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].axvline(catboost_pred['y'].mean(), color='red', linestyle='--', label=f'Mean: {catboost_pred["y"].mean():.2f}')
    axes[1, 0].legend()
    
    axes[1, 1].hist(lstm_pred['y'], bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[1, 1].set_title('LSTM Y Predictions')
    axes[1, 1].set_xlabel('Y Coordinate')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].axvline(lstm_pred['y'].mean(), color='red', linestyle='--', label=f'Mean: {lstm_pred["y"].mean():.2f}')
    axes[1, 1].legend()
    
    axes[1, 2].hist(ensemble_pred['y'], bins=50, alpha=0.7, color='purple', edgecolor='black')
    axes[1, 2].set_title('Ensemble Y Predictions')
    axes[1, 2].set_xlabel('Y Coordinate')
    axes[1, 2].set_ylabel('Frequency')
    axes[1, 2].axvline(ensemble_pred['y'].mean(), color='red', linestyle='--', label=f'Mean: {ensemble_pred["y"].mean():.2f}')
    axes[1, 2].legend()
    
    plt.tight_layout()
    plt.savefig('prediction_distributions.png', dpi=300, bbox_inches='tight')
    print("Saved: prediction_distributions.png")
    plt.close()

def plot_model_differences(catboost_pred, lstm_pred):
    """Visualize differences between model predictions"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('Model Prediction Differences', fontsize=16, fontweight='bold')
    
    # Calculate differences
    x_diff = catboost_pred['x'].values - lstm_pred['x'].values
    y_diff = catboost_pred['y'].values - lstm_pred['y'].values
    euclidean_diff = np.sqrt(x_diff**2 + y_diff**2)
    
    # X difference
    axes[0].hist(x_diff, bins=50, alpha=0.7, color='orange', edgecolor='black')
    axes[0].set_title(f'X Coordinate Difference\n(CatBoost - LSTM)')
    axes[0].set_xlabel('Difference (yards)')
    axes[0].set_ylabel('Frequency')
    axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[0].text(0.02, 0.98, f'Mean: {x_diff.mean():.3f}\nStd: {x_diff.std():.3f}', 
                 transform=axes[0].transAxes, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Y difference
    axes[1].hist(y_diff, bins=50, alpha=0.7, color='cyan', edgecolor='black')
    axes[1].set_title(f'Y Coordinate Difference\n(CatBoost - LSTM)')
    axes[1].set_xlabel('Difference (yards)')
    axes[1].set_ylabel('Frequency')
    axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
    axes[1].text(0.02, 0.98, f'Mean: {y_diff.mean():.3f}\nStd: {y_diff.std():.3f}', 
                 transform=axes[1].transAxes, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Euclidean distance
    axes[2].hist(euclidean_diff, bins=50, alpha=0.7, color='magenta', edgecolor='black')
    axes[2].set_title('Euclidean Distance\nBetween Predictions')
    axes[2].set_xlabel('Distance (yards)')
    axes[2].set_ylabel('Frequency')
    axes[2].text(0.02, 0.98, f'Mean: {euclidean_diff.mean():.3f}\nStd: {euclidean_diff.std():.3f}', 
                 transform=axes[2].transAxes, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig('model_differences.png', dpi=300, bbox_inches='tight')
    print("Saved: model_differences.png")
    plt.close()

def plot_field_heatmap(predictions, title="Prediction Heatmap"):
    """Create heatmap of predictions on NFL field"""
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Create 2D histogram
    heatmap, xedges, yedges = np.histogram2d(
        predictions['x'], predictions['y'],
        bins=[60, 27],
        range=[[0, 120], [0, 53.3]]
    )
    
    # Plot heatmap
    im = ax.imshow(heatmap.T, origin='lower', aspect='auto', 
                   extent=[0, 120, 0, 53.3], cmap='YlOrRd', interpolation='bilinear')
    
    # Add field markings
    for x in range(0, 121, 10):
        ax.axvline(x, color='white', alpha=0.3, linestyle='--', linewidth=0.5)
    ax.axhline(26.65, color='white', alpha=0.5, linestyle='-', linewidth=1)
    
    # Labels and title
    ax.set_xlabel('Field Length (yards)', fontsize=12)
    ax.set_ylabel('Field Width (yards)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Prediction Density', rotation=270, labelpad=20)
    
    plt.tight_layout()
    filename = title.lower().replace(' ', '_') + '.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Saved: {filename}")
    plt.close()

def plot_ensemble_weights_impact(catboost_pred, lstm_pred):
    """Visualize how different ensemble weights affect predictions"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    fig.suptitle('Ensemble Weight Sensitivity Analysis', fontsize=16, fontweight='bold')
    
    weights_to_test = np.arange(0, 1.1, 0.1)
    
    # Calculate RMSE for different weights (using model variance as proxy)
    rmse_x = []
    rmse_y = []
    total_rmse = []
    
    for w_cat in weights_to_test:
        w_lstm = 1 - w_cat
        ens_x = catboost_pred['x'] * w_cat + lstm_pred['x'] * w_lstm
        ens_y = catboost_pred['y'] * w_cat + lstm_pred['y'] * w_lstm
        
        # Use variance as proxy for error
        var_x = np.var(ens_x)
        var_y = np.var(ens_y)
        rmse_x.append(np.sqrt(var_x))
        rmse_y.append(np.sqrt(var_y))
        total_rmse.append(np.sqrt(var_x + var_y))
    
    # Plot X coordinate impact
    axes[0, 0].plot(weights_to_test, rmse_x, 'b-o', linewidth=2, markersize=6)
    axes[0, 0].axvline(0.6, color='red', linestyle='--', label='Selected Weight (0.6)')
    axes[0, 0].set_xlabel('CatBoost Weight', fontsize=11)
    axes[0, 0].set_ylabel('X Coordinate Std Dev', fontsize=11)
    axes[0, 0].set_title('X Coordinate Sensitivity')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    
    # Plot Y coordinate impact
    axes[0, 1].plot(weights_to_test, rmse_y, 'g-o', linewidth=2, markersize=6)
    axes[0, 1].axvline(0.6, color='red', linestyle='--', label='Selected Weight (0.6)')
    axes[0, 1].set_xlabel('CatBoost Weight', fontsize=11)
    axes[0, 1].set_ylabel('Y Coordinate Std Dev', fontsize=11)
    axes[0, 1].set_title('Y Coordinate Sensitivity')
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].legend()
    
    # Plot total impact
    axes[1, 0].plot(weights_to_test, total_rmse, 'purple', linewidth=2, marker='o', markersize=6)
    axes[1, 0].axvline(0.6, color='red', linestyle='--', label='Selected Weight (0.6)')
    axes[1, 0].set_xlabel('CatBoost Weight', fontsize=11)
    axes[1, 0].set_ylabel('Combined Std Dev', fontsize=11)
    axes[1, 0].set_title('Overall Sensitivity')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].legend()
    
    # Show weight comparison table
    axes[1, 1].axis('off')
    table_data = [
        ['Weight', 'X Std', 'Y Std', 'Total'],
        ['0.5/0.5', f'{rmse_x[5]:.3f}', f'{rmse_y[5]:.3f}', f'{total_rmse[5]:.3f}'],
        ['0.6/0.4', f'{rmse_x[6]:.3f}', f'{rmse_y[6]:.3f}', f'{total_rmse[6]:.3f}'],
        ['0.7/0.3', f'{rmse_x[7]:.3f}', f'{rmse_y[7]:.3f}', f'{total_rmse[7]:.3f}']
    ]
    table = axes[1, 1].table(cellText=table_data, cellLoc='center', loc='center',
                             colWidths=[0.2, 0.2, 0.2, 0.2])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    for i in range(len(table_data[0])):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    plt.tight_layout()
    plt.savefig('ensemble_weights_analysis.png', dpi=300, bbox_inches='tight')
    print("Saved: ensemble_weights_analysis.png")
    plt.close()

def create_summary_report(catboost_pred, lstm_pred, ensemble_pred):
    """Create a comprehensive summary report"""
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    fig.suptitle('NFL Big Data Bowl 2026 - Ensemble Model Summary Report', 
                 fontsize=18, fontweight='bold', y=0.98)
    
    # Statistics summary
    ax_stats = fig.add_subplot(gs[0, :])
    ax_stats.axis('off')
    
    stats_text = f"""
    MODEL STATISTICS SUMMARY
    
    CatBoost:  X: [{catboost_pred['x'].min():.2f}, {catboost_pred['x'].max():.2f}] μ={catboost_pred['x'].mean():.2f} σ={catboost_pred['x'].std():.2f}
                    Y: [{catboost_pred['y'].min():.2f}, {catboost_pred['y'].max():.2f}] μ={catboost_pred['y'].mean():.2f} σ={catboost_pred['y'].std():.2f}
    
    LSTM:           X: [{lstm_pred['x'].min():.2f}, {lstm_pred['x'].max():.2f}] μ={lstm_pred['x'].mean():.2f} σ={lstm_pred['x'].std():.2f}
                    Y: [{lstm_pred['y'].min():.2f}, {lstm_pred['y'].max():.2f}] μ={lstm_pred['y'].mean():.2f} σ={lstm_pred['y'].std():.2f}
    
    Ensemble:    X: [{ensemble_pred['x'].min():.2f}, {ensemble_pred['x'].max():.2f}] μ={ensemble_pred['x'].mean():.2f} σ={ensemble_pred['x'].std():.2f}
                    Y: [{ensemble_pred['y'].min():.2f}, {ensemble_pred['y'].max():.2f}] μ={ensemble_pred['y'].mean():.2f} σ={ensemble_pred['y'].std():.2f}
    
    Total Predictions: {len(ensemble_pred):,} | Weights: CatBoost={Config.ENSEMBLE_WEIGHTS['catboost']}, LSTM={Config.ENSEMBLE_WEIGHTS['lstm']}
    """
    ax_stats.text(0.1, 0.5, stats_text, fontsize=11, family='monospace',
                  verticalalignment='center',
                  bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
    
    # Scatter plots
    ax1 = fig.add_subplot(gs[1, 0])
    sample_size = min(1000, len(catboost_pred))
    sample_indices = np.random.choice(len(catboost_pred), sample_size, replace=False)
    ax1.scatter(catboost_pred.iloc[sample_indices]['x'], 
                catboost_pred.iloc[sample_indices]['y'], 
                alpha=0.4, s=10, c='blue', label='CatBoost')
    ax1.set_xlabel('X Coordinate')
    ax1.set_ylabel('Y Coordinate')
    ax1.set_title('CatBoost Predictions\n(sample)')
    ax1.set_xlim(0, 120)
    ax1.set_ylim(0, 53.3)
    ax1.grid(True, alpha=0.3)
    
    ax2 = fig.add_subplot(gs[1, 1])
    ax2.scatter(lstm_pred.iloc[sample_indices]['x'], 
                lstm_pred.iloc[sample_indices]['y'], 
                alpha=0.4, s=10, c='green', label='LSTM')
    ax2.set_xlabel('X Coordinate')
    ax2.set_ylabel('Y Coordinate')
    ax2.set_title('LSTM Predictions\n(sample)')
    ax2.set_xlim(0, 120)
    ax2.set_ylim(0, 53.3)
    ax2.grid(True, alpha=0.3)
    
    ax3 = fig.add_subplot(gs[1, 2])
    ax3.scatter(ensemble_pred.iloc[sample_indices]['x'], 
                ensemble_pred.iloc[sample_indices]['y'], 
                alpha=0.4, s=10, c='purple', label='Ensemble')
    ax3.set_xlabel('X Coordinate')
    ax3.set_ylabel('Y Coordinate')
    ax3.set_title('Ensemble Predictions\n(sample)')
    ax3.set_xlim(0, 120)
    ax3.set_ylim(0, 53.3)
    ax3.grid(True, alpha=0.3)
    
    # Model agreement analysis
    ax4 = fig.add_subplot(gs[2, :])
    x_diff = np.abs(catboost_pred['x'].values - lstm_pred['x'].values)
    y_diff = np.abs(catboost_pred['y'].values - lstm_pred['y'].values)
    total_diff = np.sqrt(x_diff**2 + y_diff**2)
    
    ax4.hist(total_diff, bins=50, alpha=0.7, color='coral', edgecolor='black')
    ax4.axvline(total_diff.mean(), color='red', linestyle='--', linewidth=2, 
                label=f'Mean Disagreement: {total_diff.mean():.2f} yards')
    ax4.axvline(np.median(total_diff), color='blue', linestyle='--', linewidth=2, 
                label=f'Median Disagreement: {np.median(total_diff):.2f} yards')
    ax4.set_xlabel('Euclidean Distance Between Models (yards)', fontsize=11)
    ax4.set_ylabel('Frequency', fontsize=11)
    ax4.set_title('Model Agreement Analysis', fontsize=13, fontweight='bold')
    ax4.legend(fontsize=10)
    ax4.grid(True, alpha=0.3)
    
    plt.savefig('comprehensive_summary.png', dpi=300, bbox_inches='tight')
    print("Saved: comprehensive_summary.png")
    plt.close()

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    print("=" * 70)
    print("NFL BIG DATA BOWL 2026 - ADVANCED ENSEMBLE WITH VISUALIZATIONS")
    print("=" * 70)
    
    # Load data
    test_input = pd.read_csv(Config.DATA_DIR / "test_input.csv")
    test_template = pd.read_csv(Config.DATA_DIR / "test.csv")
    print(f"\nData loaded: {test_input.shape[0]} input rows, {test_template.shape[0]} predictions needed")
    
    # Load models
    print("\n" + "="*70)
    models_x_cat, models_y_cat, features_cat = load_catboost_models(Config.CATBOOST_MODEL_PATH)
    print(f"✓ CatBoost: {len(models_x_cat)} X models, {len(models_y_cat)} Y models, {len(features_cat)} features")
    
    models_lstm, scalers_lstm = load_lstm_models(Config.LSTM_MODEL_DIR, Config.LSTM_N_FOLDS)
    print(f"✓ LSTM: {len(models_lstm)} models loaded")
    
    # Generate predictions
    print("\n" + "="*70)
    catboost_pred = predict_catboost(models_x_cat, models_y_cat, features_cat, test_input, test_template)
    print(f"✓ CatBoost: {catboost_pred.shape[0]} predictions")
    print(f"  X: [{catboost_pred['x'].min():.2f}, {catboost_pred['x'].max():.2f}]")
    print(f"  Y: [{catboost_pred['y'].min():.2f}, {catboost_pred['y'].max():.2f}]")
    
    lstm_pred = predict_lstm(models_lstm, scalers_lstm, test_input, test_template)
    print(f"✓ LSTM: {lstm_pred.shape[0]} predictions")
    print(f"  X: [{lstm_pred['x'].min():.2f}, {lstm_pred['x'].max():.2f}]")
    print(f"  Y: [{lstm_pred['y'].min():.2f}, {lstm_pred['y'].max():.2f}]")
    
    # Create ensemble (choose method)
    print("\n" + "="*70)
    print("Creating ensemble with 60/40 weighting (CatBoost/LSTM)...")
    
    # Method 1: Position-adaptive ensemble
    ensemble_pred = create_position_adaptive_ensemble(
        catboost_pred, lstm_pred, test_input, Config.POSITION_WEIGHTS
    )
    
    # Method 2: Confidence-weighted (alternative - comment out if not using)
    # ensemble_pred = create_confidence_weighted_ensemble(
    #     catboost_pred, lstm_pred, Config.ENSEMBLE_WEIGHTS
    # )
    
    print(f"✓ Ensemble: {ensemble_pred.shape[0]} predictions")
    print(f"  X: [{ensemble_pred['x'].min():.2f}, {ensemble_pred['x'].max():.2f}]")
    print(f"  Y: [{ensemble_pred['y'].min():.2f}, {ensemble_pred['y'].max():.2f}]")
    
    # Generate visualizations
    print("\n" + "="*70)
    print("Generating visualizations...")
    
    plot_prediction_distributions(catboost_pred, lstm_pred, ensemble_pred)
    plot_model_differences(catboost_pred, lstm_pred)
    plot_field_heatmap(catboost_pred, "CatBoost Prediction Heatmap")
    plot_field_heatmap(lstm_pred, "LSTM Prediction Heatmap")
    plot_field_heatmap(ensemble_pred, "Ensemble Prediction Heatmap")
    plot_ensemble_weights_impact(catboost_pred, lstm_pred)
    create_summary_report(catboost_pred, lstm_pred, ensemble_pred)
    
    print("✓ All visualizations generated")
    
    # Save submission
    print("\n" + "="*70)
    submission = ensemble_pred[['id', 'x', 'y']].copy()
    submission.to_csv('submission.csv', index=False)
    print(f"✓ Submission saved: {len(submission)} predictions")
    print(f"✓ No NaN values: {submission.isnull().sum().sum() == 0}")
    print("=" * 70)
    
    # Print summary statistics
    print("\nFINAL SUMMARY:")
    print(f"  Total predictions: {len(submission):,}")
    print(f"  X range: [{submission['x'].min():.2f}, {submission['x'].max():.2f}]")
    print(f"  Y range: [{submission['y'].min():.2f}, {submission['y'].max():.2f}]")
    print(f"  Ensemble method: Position-adaptive with 60/40 base weights")
    print("=" * 70)

if __name__ == "__main__":
    main()